In [1]:
import numpy as np

import matplotlib.pyplot as plt
from ipywidgets import interactive, interact
from matplotlib import animation, rc
import ipywidgets as widgets
%matplotlib widget
from scipy import fftpack as fft
from scipy import signal as sig
plt.close('all')

In [2]:
#Import Data
raw0 = np.fromfile(open("open_press.smpl"), dtype=np.complex64)

#The FMCOMMS3 uses an automatic gain control. Haven't figured out how to config manual gain control correctly so I need to remove pre-gain correction.

#Constants
fs = 1000000 # Samples/seconds
#Note: set gain to ~45dB and configure gr-iio to manual gain mode in GRC TODO: nake this work.

#raw0 with respect to time
Traw = np.linspace(0.0, raw0.size/fs, raw0.size)

In [3]:
#Lets view raw signal and its FT

def calcFFT(data, fs):
    yf = fft.fft(data)
    yf = fft.fftshift(yf)
    xf = fft.fftfreq(data.size, 1/fs)
    xf = fft.fftshift(xf)
    return [xf, yf]

plt.figure()
plt.plot(Traw, raw0.real, Traw, raw0.imag)
plt.show()

rawFFT = calcFFT(raw0, fs)

plt.figure()
plt.plot(rawFFT[0], rawFFT[1].real, rawFFT[0], rawFFT[1].imag)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
#TODO: To optimize integrate this into find packets method
#isolate noisy start/where gain is not set correctly
sos = sig.butter(15, 100, 'lp', fs=fs, output='sos')
rawf = sig.sosfilt(sos, raw0)

gain_set =abs(rawf)>.005  

gain_set_loc = np.argmax(gain_set)
raw = raw0[gain_set_loc:]

#Show successful isolation. New dataset is in orange
plt.figure("Remove FMCOMMS Gain Instability")
plt.plot(Traw, raw0.real, Traw[gain_set_loc:], raw.real)
plt.show()

T = np.linspace(0.0, raw.size/fs, raw.size)
#Plot with new time reference
plt.figure("Received Broadcast Plotted Over Time")
plt.plot(T, raw.real, T, raw.imag)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
#Now that we can hypothosize that the remote sends a same/similar message the longer the button is pressed, we can 
#isolatethe message based on the first occurence 
#Maybe optimal method is to use derivative like what is used to isolate bits

def isolatePacket(raw_data):
    SMPL_BUFF = 2000 # buffer to 
    PCKT_SIZE_EST = 50000
    tx =abs(raw_data)>.05  
    msg_loc = np.where(tx == True)[0] #locatoins where value non-zero
    start_tx = msg_loc[0] - SMPL_BUFF #I know this is dangerous but I'm a badass
    
    #find end of packet
    diff_msg_loc = np.roll(msg_loc, 1)
    diff_msg_loc[0] = msg_loc[0]
    temp = np.where((msg_loc - diff_msg_loc) > PCKT_SIZE_EST)[0]
    end_tx = msg_loc[temp[0]] - PCKT_SIZE_EST + SMPL_BUFF
    
    return [start_tx, end_tx]


pkt_size = isolatePacket(raw)
raw_pkt = raw[pkt_size[0]:pkt_size[1]]
x_raw_pkt = T[pkt_size[0]:pkt_size[1]] #For plotting in time
#x_raw_pkt = range(raw_pkt.real.size)

print("PKT Start in time:", T[pkt_size[0]+2000])
print("PKT End in time is about:", T[pkt_size[1]-2000])

plt.figure("Isolated Packet")
plt.plot(x_raw_pkt, raw_pkt.real, x_raw_pkt, raw_pkt.imag)
plt.show()

#Find dominant frequency/ carrier freq? Not sure if that's the right term
raw_pktFFT = calcFFT(raw_pkt, fs)
temp = np.max(np.abs(raw_pktFFT[1]))
F0_index = np.where(np.abs(raw_pktFFT[1]) == temp)
F0 = raw_pktFFT[0][(F0_index[0])][0]
print("Dominant Freq is:", F0)

PKT Start in time: 0.044438011136349743
PKT End in time is about: 0.0941330235901258


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Dominant Freq is: 5791.973181860509


## Initial Analysis of packets
### Packet Makeup
Each packet seems to consist of a preamble followed by a message body. In phase (I) and quadrature (Q) seems to be out of phase by a consistent amount. Looks like 90 degree phase difference. 
#### Preamble Data
The preamble consists of 6 pulses.

| Pulse # | Periods | Positive Phase Angle (<180deg) | Time difference b/t I and Q | Pulse Frequency |
| ------- | ------- | ------------------------------ | --------------------------- | --------------- |
| 1       | 1.5     | +                              | .0000376                    |
| 2       | 3       | +                              | .0000439                    |
| 3       | 2.5     | +                              | .0000461                    |
| 4       | 2.5     | -                              | .0000438                    | 5792 |
| 5       | 3       | -                              | .0000439                    | 5792 |
| 6       | 1.5     | +                              | .0000427                    | 5733 |


Average time difference in phase b/t I and Q is 440.8 (excluding Pulse 1 outlier). Frequency of pulses appears to be ~5792 Hz. I stopped measuring because it's smarter to just look at where the max FFT value occurs. Max off FFT put dominant freq at 5.791 kHz. This is in line with manual calculations. The phase can difference b/t I and Q data is estimated to be 91.9 degrees. Basically 90 degrees. I guess this should've been assumed because of the definition of quadrature. 

If the protocol is anything like CAN, SPI, I2C, I assume that the preamble contains some sort of address.

#### Body Data
Body conisists of 58 pulses. Each pulse seems to vary in number of periods. In addition, the length of time b/t each pulse seems to vary as well. Table includes first 6 pulses. I don't feel like manually categorizing them all.

| Pulse # | Periods | Positive Phase Angle (<180deg) |
| ------- | ------- | ------------------------------ |
| 1       | 1.5     | -                              |
| 2       | 3       | -                              |
| 3       | 2.5     | -                              |
| 4       | 3       | +                              |
| 5       | 3       | -                              |
| 6       | 1.5     | +                              |

#### Hypotheses
* BPSK: Bits values are encoded into the phase of the incoming signal
    * The phase differences allude to a BPSK scheme
* ASK (Amplitude Shift-Keying): Bit values are determined by time, similar to electrical signals. 
    * The periodic differences allude to time encoding method
    * TBH I think that this is the most likely cast given that a car key is a low cost device. It seems cheapest to decode.
    * ON/OFF keying
    * this also accounts for the differences in phases. 

### Decoding Courses of Action
How do I prove / disprove each hypothesis?
#### ASK
Since this is the most likely COA, it will be the first I explore
1. Determine duration of each pulse/bit
1. Get bit values
    * Integrate and Dump
    * LPF the entire thing and then use bit timing
    * LPF and identify time intervals.
    * I guess both ways mentioned above are essentially the same. LPF and integrate are basically the same thing I'm pretty sure. It just comes down to which is less computationally intensive. (Kids this is why you don't join the army. It makes you stupid and forget things you learnt sophomore year of college.)
1. Compare periods/time of each pulse accross multiple packets of the same button press
1. Compare periods/time of each pulse accross multiple packets with different button presses

#### BPSK
1. Determine phase of each pulse
1. Compare phase accross multiple packets of the same button press
1. Compare phase accross multiple packets with different button presses



In [45]:
raw2_pkt = -raw_pkt*raw_pkt
raw2_pkt.imag = -raw_pkt.imag*raw_pkt.imag
plt.figure()
plt.plot(x_raw_pkt, raw2_pkt.real, x_raw_pkt, raw2_pkt.imag) #x_raw_pkt, abs(raw2_pkt))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
#Isolate bits
#maybe another/smarter way to do this is to use the derivative
#Based on graphical observation of the graph above, I found 4 bit lengths in # of samples. I will average to get an
#    estimated bit length

bLeng_raw = [9.415e5-4.205e5, 9.411e5-9.404e5, 9.431e5-9.425e5, 9.441e5-9.435e5]
BLENG_EST = int (np.sum(bLeng_raw) / len(bLeng_raw))
BLENG_EST = 400

ZERO_TRESH = .05    #Determined graphically. Threshold to determine b/t zero and non-zero
SMPL_BUFF = 100 # buffer to add at the end of the bit for optimal viewing

#Method 1 using zero threshold
#Find start of bit
tx =abs(raw_pkt)>ZERO_TRESH 
bit_loc = np.where(tx == True)[0] #locations where value non-zero
start_tx = bit_loc[0] - SMPL_BUFF #I know this is dangerous but I'm a badass
#Find end of bit
diff_msg_loc = np.roll(bit_loc, 1)
diff_msg_loc[0] = bit_loc[0]
temp = np.where((bit_loc - diff_msg_loc) > BLENG_EST)[0]
end_tx = bit_loc[temp[0]] #- BLENG_EST + SMPL_BUFF

print(start_tx, end_tx)
raw_bit = raw_pkt[start_tx:end_tx]
x_raw_bit = x_raw_pkt[start_tx:end_tx]



plt.figure()
plt.plot(x_raw_bit, raw_bit.real, x_raw_bit, raw_bit.imag)
plt.show()

plt.figure()
plt.xlim(-1,1)
plt.ylim(-1,1)
symb = np.sum(raw_bit)
symb = symb / abs(symb)
plt.scatter(symb.real, symb.imag)
plt.show()


print("Bit Start in time:", x_raw_pkt[start_tx+SMPL_BUFF])
print("Bit End in time:", x_raw_pkt[end_tx-SMPL_BUFF])
   

1900 2732


<ipython-input-45-5c6c69f82f93>:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-45-5c6c69f82f93>:34: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Bit Start in time: 0.044438011136349743
Bit End in time: 0.0450700112947316


In [8]:
#I want to see what this looks like in freq domain
N = 630000000
fft_pkt = fft.fft(raw_pkt)
#fft_pkt = fft.fft(np.cos(2*np.pi*T*5000))


plt.figure(9)
xf = fft.fftfreq(raw_pkt.size, d=1/fs)
plt.plot(xf, fft_pkt)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def isolateBitThresh(pkt):
    BLENG_EST = 500     #Bit length estimate in samples
    ZERO_TRESH = .05    #Determined graphically. Threshold to determine b/t zero and non-zero
    SMPL_BUFF = 0 # buffer to add at the end of the bit for optimal viewing

    #pkt = pkt[start:]    #TODO needs to be made more efficient
    tx =abs(pkt)>ZERO_TRESH 
    bit_loc = np.where(tx == True)[0] #locations where value non-zero
    start_tx = bit_loc - SMPL_BUFF #I know this is dangerous but I'm a badass
    end_tx = bit_loc + BLENG_EST + SMPL_BUFF
    bit_bounds = [start_tx, end_tx]
    return (start_tx, end_tx)


In [17]:
#Get symbol bounds
bounds = isolateBitThresh(raw_pkt)

#Get Symbols
symb = np.empty(bounds[0].size, dtype=complex)
for i in range(bounds[0].size):
    #integrate and dump
    #print(raw_pkt[bounds[0][i]:bounds[1][i]])
    temp = raw_pkt[bounds[0][i]:bounds[1][i]]
    symb[i] = np.sum(temp)
    symb[i] = symb[i] / abs(symb[i])

print(np.sum(raw_pkt[bounds[0][i]:bounds[1][i]]))
#By examining bounds, it is apparent that the function isolateBitThreshold doesn't do a good job identifying
#the bit start locations.
%matplotlib notebook
plt.figure(6)
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
for i in range(10):
    print("Angle: ", np.angle(symb[i], deg=True), "Raw: ", symb[i])
    plt.scatter(symb[i].real, symb[i].imag)
plt.show()

plt.figure(7)
const = 10
for i in range(9):
    plt_num = 9*100 + 10 + i+1
    plt.subplot(plt_num)
    symbs = raw_pkt[bounds[0][i+const]:bounds[1][i+const]]
    print(symbs.size)
    plt.plot(range(symbs.size), symbs.real, range(symbs.size), symbs.imag)

(-0.9379883+0.8691406j)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Angle:  110.23011228843018 Raw:  (-0.3457913843470033+0.9383114187258851j)
Angle:  110.58534742800123 Raw:  (-0.35160225313130766+0.9361494835724622j)
Angle:  111.72807253201772 Raw:  (-0.3702019489166574+0.928951299594499j)
Angle:  112.9360210107854 Raw:  (-0.38970300886841946+0.9209405870515756j)
Angle:  114.19193980983528 Raw:  (-0.4097947155949258+0.9121777738305588j)
Angle:  115.35893291483785 Raw:  (-0.4282875524223257+0.9036425025639805j)
Angle:  116.58599140756407 Raw:  (-0.447540457268454+0.8942636854462688j)
Angle:  117.77181128603488 Raw:  (-0.465951382571781+0.8848103237866553j)
Angle:  118.98829237014793 Raw:  (-0.4846308932316615+0.8747187532718628j)
Angle:  120.15897485191037 Raw:  (-0.5024009763131574+0.8646347546794464j)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

500
500
500
500
500
500
500
500
500


In [8]:
#Using a loop method to do integrate and dump based on the bit sizes
def isolateBitThreshLoop(pkt):
    BLENG_EST = 500     #Bit length estimate in samples
    ZERO_TRESH = .05    #Determined graphically. Threshold to determine b/t zero and non-zero
    SMPL_BUFF = 6 # buffer to add at the end of the bit for optimal viewing
    
    tx =abs(pkt)>ZERO_TRESH
    symbs = np.empty([int (tx.size/BLENG_EST), BLENG_EST], dtype=complex)
    #print(pkt[0:0+BLENG_EST])
    x = 0; #symbs array iterator
    i = 0
    while tx.size > i:
        if (True == tx[i]):
            #symbs[x] = np.sum(pkt[i:i+BLENG_EST])
            #symbs[x] = symbs[x] / abs(symbs[x])
            symbs[x] = pkt[i:i+BLENG_EST]
            
            x += 1
            i = i + BLENG_EST + SMPL_BUFF
            
        else:
            i += 1
    return symbs

symbs = isolateBitThreshLoop(raw_pkt)

plt.figure(7)
plt.xlim(-2, 2)
plt.ylim(-2, 2)
for i in range(9):
    symb = np.sum(symbs[i])
    symb = symb / abs(symb)
    print("Angle: ", np.angle(symb, deg=True), "Raw: ", symb)
    plt.scatter(symb.real, symb.imag)
plt.show()

plt.figure(8)
const = 10
for i in range(9):
    plt_num = 9*100 + 10 + i+1
    plt.subplot(plt_num)
    plt.plot(range(symbs[i+const].size), symbs[i+const].real, range(symbs[i+const].size), symbs[i+const].imag)


print(np.where(symbs==False))

Angle:  110.23011228843018 Raw:  (-0.3457913843470033+0.9383114187258851j)
Angle:  -71.55101306406519 Raw:  (0.31646019464420316-0.948605790202523j)
Angle:  178.54471131225134 Raw:  (-0.9996774480304919+0.025396848214752216j)
Angle:  54.00792644197148 Raw:  (0.5876733252261878+0.8090983023203024j)
Angle:  -56.75316871733398 Raw:  (0.54824697825146-0.8363164776794387j)
Angle:  117.31202311974425 Raw:  (-0.4588360145138036+0.8885209686805869j)
Angle:  -114.50006288871356 Raw:  (-0.4146942414432332-0.9099608157024243j)
Angle:  58.99619283344329 Raw:  (0.5150950304757114+0.8571330757701665j)
Angle:  -53.243270496359116 Raw:  (0.5984187053027566-0.8011835327462568j)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  6,  11,  11, ..., 106, 106, 106]), array([268, 352, 377, ..., 497, 498, 499]))


In [19]:
#LPF fx. 5th order BW filter
#f_cutoff is the cutoff freq for the lpf
def lpf(data, fs, f_cutoff):
    sos = sig.butter(15, f_cutoff, 'lp', fs=fs, output='sos')
    filtered = sig.sosfilt(sos, raw0)

    return filtered

In [51]:
rawf = lpf(raw, fs, 45000)
print(rawf.size, raw.size)
#Tpkt = np.linspace(0.0, rawf_pkt.size/fs, rawf_pkt.size)
plt.figure()
plt.plot(T, rawf.real+rawf.imag, T, raw.real+raw.imag) #Tpkt, raw_pkt.real+raw_pkt.imag)
plt.show()

print(.0449294-.044953)
print(.0450475-.0450822)
print(.0451322-.045167)
print(.0451753-.0452184)


3791461 3791461


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-2.3599999999998622e-05
-3.470000000000556e-05
-3.4800000000001496e-05
-4.309999999999731e-05


In [49]:
raw_abs = np.abs(raw)
plt.figure()
plt.plot(T, raw_abs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function matplotlib.pyplot.show(*args, **kw)>

In [5]:
rawp = raw.real + raw.imag


plt.figure()
plt.plot(T, rawp, T, raw.real, T, raw.imag)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
#Method 2 of bit isolations using derivative
#I have abandoned method 2 because it took too much time and method 1 works ok.
# added a LPF IOT to make results more consistent

DIFF_TRESH = .004

filt_pkt = lpf(raw_pkt, fs, 246)
diff_pkt = np.roll(filt_pkt, -1)    #diff_pkt is a copy of raw_pkt shifted 1 to the left

diff_pkt[-1] = filt_pkt[-1] 
diff = filt_pkt - diff_pkt


x_diff = range(diff.size)

%matplotlib notebook
plt.figure(6)
plt.plot(x_diff, diff.real)
plt.show()

tx =abs(diff.real)>=DIFF_TRESH 

bit_loc = np.where(tx == True)[0] #locations where value non-zero
print(bit_loc[0:20])
start_bit = bit_loc[0] 
end_bit = bit_loc[1] + 2

print(raw_pkt[1999:2010])
print(diff_pkt[1999:2010])
print(diff[1990:2010])
print(bit_loc[0])

plt.figure(9)
plt.plot(range(raw_pkt[1999:2010].size), raw_pkt[1999:2010].real, label='raw_pkt-real')
plt.plot(range(raw_pkt[1999:2010].size), raw_pkt[1999:2010].imag, label='raw_pkt-imag')
plt.plot(range(diff_pkt[1999:2010].size), diff_pkt[1999:2010].real, label='diff_pkt-real')
plt.plot(range(diff_pkt[1999:2010].size), diff_pkt[1999:2010].real, label='diff_pkt-real')
plt.plot(range(diff_pkt[1999:2010].size), diff_pkt[1999:2010].imag, label='diff_pkt-imag')
plt.plot(range(diff[1990:2010].size), diff[1990:2010].real, label='diff-real')
plt.plot(range(diff[1990:2010].size), diff[1990:2010].imag, label='diff-imag')
plt.legend()
plt.show()

print(raw_pkt[2000])
print(diff_pkt[2000])
print(diff[2000])

bit_msg = filt_pkt[start_bit:end_bit]
x_diff = range(bit_msg.size)

plt.figure(7)
plt.plot(x_diff, bit_msg.real, x_diff, bit_msg.imag)
plt.show()

plt.figure(8)
plt.xlim(-1,1)
plt.ylim(-1,1)
symb = np.sum(raw_bit)
symb = symb / abs(symb)
plt.scatter(symb.real, symb.imag)
plt.show

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [18]:
#Understanding Quadrature Demod

freq = 10
t = np.arange(0, 4, .01)
x = np.cos(freq*t + 1*np.pi/4)
x2 =np.cos(freq*t)
x3 = np.cos(freq*t + np.pi/4)
x4 = np.cos(freq*t + np.pi/2)
x5 = np.cos(freq*t + 3*np.pi/4)
plt.figure()
#Plot original signal
plt.subplot(411)
plt.plot(t, x2, t, x3, t, x4, t, x5)

#Implement Quadrature demod
comp_data = x*((np.cos(freq*t)) - 1j*np.sin(freq*t))
plt.subplot(412)
plt.plot(t, comp_data.real, t, comp_data.imag)

#Plot Symbol
plt.subplot(413)
plt.xlim(-1,1)
plt.ylim(-1,1)
#symb = comp_data
symb = np.sum(comp_data)
symb = symb / abs(symb)
plt.scatter(symb.real, symb.imag)

plt.subplot(414)
plt.plot(t, comp_data.real+comp_data.imag, t, x)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …